# HDS5210-2022 Fall Midterm

In the midterm exercise, you're going to use all the programming and data management skills you've developed so far to build a pricing calculator that will calculate how much money should be reimbursed for the visits in a CSV file. To do this, you will need to get allowed amounts (aka rates) from a JSON file, apply some special rules, and then calculate various totals by hospital or by month.

Each step of the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure you're getting each step correct.

All functions require docstrings with a description and at least one test case.

The midterm is due Monday, October 24th at 11:59 PM CST.

---

## Step 1: Average Rate

With the `/data/negotiated_rates.json` file as input for your first function, read all the `allowed_amount` attributes and calculate an average allowed amount over all rates in the file.

Your function should be named **average_rate()**, take the file's name as it's input parameter, and return a float as the result.

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [1]:
import json
from pathlib import Path
file='/data/negotiated_rates.json'
def average_rate(file):
    """(str) -> float
    This function iterates the allowed amounts from the negotiated rates json file
    and adds them up to a total to calcuate the average allowed amount in the file.
    
    This function will expect the allowed amounts to be float values. The average is
    calculated by dividing the total allowed amounts over the count of allowed amounts.
    The final result of the calculation will be rounded to two decimal places.
    
    >>> average_rate(file)
    38.67
    """
    negotiated_rates = json.load(open(file))
    
    amounts_total = 0
    amounts_count = 0

    for outta_network in negotiated_rates['out_of_network']:
        for allowed_amounts in outta_network['allowed_amounts']:
            for amts in allowed_amounts['payments']:
                values = allowed_amounts['payments']['allowed_amount']
            amounts_total += values
            amounts_count += 1

    average = round((amounts_total / amounts_count),2)
    return(average)  

In [2]:
assert(average_rate('/data/negotiated_rates.json') == 38.67)

In [3]:
average_rate('/data/negotiated_rates.json')

38.67

---

## Step 2: Rate for a Billing Code / Service Code Combination

For the next step, we need to be able to look up the allowed amount for any given billing code / service code combination.

In this data, the billing code represents the service or procedure that was provided. The service code represents the type of site where the service was provided.

Your function should be named **get_rate()** and take three parameters: the JSON file name, the billing code, and the service code. If your code can't find that combination in the file, it should return None.

In [4]:
import json

from pathlib import Path
file='/data/negotiated_rates.json'
def get_rate(file, billing_code, service_code):
    """(str, str, str) -> float
    This function will take three string parameters as a combination and will output
    them as a float value which will be the allowed amount.

    The billing code represents the service/procedure and the service code represents
    the site of where the service/procedure was provided.

    If the combination of billing codes and service codes do not match, the code will
    return None.

    >>> get_rate('/data/negotiated_rates.json','G0283','11')
    8.78

    >>> get_rate('/data/negotiated_rates.json','97140','11')
    20.03

    >>> get_rate('/data/negotiated_rates.json','97110','12')
    26.62
   """
    negotiated_rates = json.load(open(file))
    output = None

    for outta_network in negotiated_rates['out_of_network']:
        b_code = outta_network['billing_code']
        for service in outta_network['allowed_amounts']:
            s_code = service['service_code']
            if billing_code == b_code and service_code == s_code:
                for allowed_amounts in outta_network['allowed_amounts']:
                    if allowed_amounts["service_code"] == service_code:
                        payments = allowed_amounts['payments']
                        values = payments['allowed_amount']
                    
            output = values                 
                
    return(float(output))

In [5]:
assert (get_rate('/data/negotiated_rates.json','G0283','11') == 8.78)
assert (get_rate('/data/negotiated_rates.json','97140','11') == 20.03)
assert (get_rate('/data/negotiated_rates.json','97110','12') == 26.62)

In [6]:
get_rate('/data/negotiated_rates.json','97140','12')

24.64

---

## Part 3: Special Rules for Rates

There are some special rules for adjusting rates depending on a patient's age and the day of week.  (These are made up. Not from the real world.) In the next part of the midterm, you'll need to create a function that applies these extra rules to a rate and return the adjusted value.

1. If the day of week is Monday, charge only 75% of the allowed amount.
2. If the patient's age is 65 or higher, charge only 50% of the allowed amount.
3. If's both Monday and the patient's age is 65 or higher, charge only 50% of the allowed amount.
4. If neither of these conditions are true, charge the whole amount.

Your function should be named **get_adjusted_rate()** and take five parameters: file name, billing code, service code, patient age, and visit date.  Your function should return the adjusted rate (based on the rules above) or None if the rate couldn't be found in the file.

Note that your function will take a date in the form `%Y-%m-%d` ([see datetime.strptime()](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)) and will need to calculate the [day of week](https://docs.python.org/3/library/datetime.html#datetime.date.weekday).

**ROUND YOUR ANSWER TO 2 DECIMAL PLACES**

In [7]:
from datetime import datetime
from pathlib import Path
file='/data/negotiated_rates.json'
def get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date):
    """(str, str, str, int, date) -> float
    This function will take the previous three parameters as well as an additional
    two (an integer and date) and use them to calculate the adjusted rate for the
    patient. 
    
    The adjusted rate is based on a set of rules that will input the patient's
    age and the date they were in the clinic. 
    
    If neither the patient's age or visit date are valid from the file, the code will return None.
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03')  
    4.39
    
    >>> get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03')
    6.58
    """
    day = visit_date
    format_date = datetime.strptime(day, '%Y-%m-%d')
    weekday = format_date.weekday()
    
    negotiated_rates = json.load(open(file))
    output = float()
    values = None
    
    for outta_network in negotiated_rates['out_of_network']:
        b_code = outta_network['billing_code']
        for service in outta_network['allowed_amounts']:
            s_code = service['service_code']
            if billing_code == b_code and service_code == s_code:
                for allowed_amounts in outta_network['allowed_amounts']:
                    if allowed_amounts["service_code"] == service_code:
                        payments = allowed_amounts['payments']
                        values = payments['allowed_amount']
    
    if values == None:
        return None
    
    if pat_age >= 65:
        output = (values * 0.5)
    elif weekday == 0:
        output = (values * 0.75)
    elif pat_age >= 65 and weekday == 0:
        output = (values * 0.5)
    else:
        output = (values * 1)
        
    return(round(output,2))

In [8]:
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-07') == 8.78)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',22,'2022-01-03') == 6.58)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-03') == 4.39)
assert (get_adjusted_rate('/data/negotiated_rates.json','G0283','11',76,'2022-01-08') == 4.39)

In [9]:
get_adjusted_rate('/data/negotiated_rates.json','G0283','12',65,'2021-06-15')

---

## Step 4: Calculate total payments for a list of visits

Last step, we're going to put your get_adjusted_rate() function to work on a list of visits from a few hospitals in the St. Louis region.

Your last function, **summarize_reimbursement()** needs to read an encounter file and the negotiated rates file, compute an adjusted rate for each encounter (row) in the input file, and return two dictionaries of information:
1. Total expected reimbursement by month
2. Total expected reimbursement by hospital

Your should be able to run your function as `by_month, by_hospital = summarize_reimbursement(visits, negotiated_rates)` and have the two answers below:

**by_month**
```json
{
    '2021-05': 192.38,
    '2021-03': 378.72,
    '2021-07': 277.67,
    '2021-06': 236.53,
    '2021-11': 229.7,
    '2021-10': 234.52,
    '2021-12': 297.87,
    '2021-04': 337.7,
    '2021-09': 160.4,
    '2021-01': 111.91,
    '2021-02': 158.55,
    '2021-08': 152.28
}
```

**by_hospital**
```json
{
    'Missouri Baptist': 514.18,
    'SSM DePaul': 460.02,
    'SLU Hospital': 409.67,
    'Barnes Jewish': 485.7,
    'Mercy Springfield': 518.59,
    'Mercy St. Louis': 380.07
}
```

**Round your totals to 2 decimal places**

**If rates are not found, just ignore them**

In [10]:
import json
import csv
from pathlib import Path
file='/data/negotiated_rates.json'
def summarize_reimbursement(visits, negotiated_rates):
    """(str, str) -> float
    This function outputs two dictionaries with computations from each row in the input files
    which act as the function's parameters. The calculated adjusted rates are summarized by
    month and by hospital and are created as the two dictionaries.
    
    This function takes the calculations from the previous adjusted rate function and creates
    a sum as the output by each month in the calendar year and by each hospital in the region.
    
    The function uses prevoius logic based on the calculation and combination of service code
    and billing code, so if they do not match within each parameter, they will not be added to
    the sum.
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07']
    277.67
    
    >>> summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield']
    518.59
    """
    
    with open('visits.csv', 'r') as visits:
        csv_reader = csv.reader(visits)

        by_month = {}
        by_hospital = {}

        for i, visit in enumerate(visits):
            if i == 0:
                continue

            parsed_visit = visit.strip().split(",")
            visit_date = parsed_visit[6]
            year_month = visit_date[0:7]
            hospital = parsed_visit[1]      
            service_code = parsed_visit[5]
            billing_code = parsed_visit[4]
            pat_age = int(parsed_visit[3])


            adjusted_rate = get_adjusted_rate(file, billing_code, service_code, pat_age, visit_date)
            
            if by_month.get(year_month) == None and adjusted_rate != None:
                by_month[year_month] = round(adjusted_rate, 2) 
            elif adjusted_rate != None:
                by_month[year_month] = round((by_month[year_month] + adjusted_rate), 2)

            if by_hospital.get(hospital) == None and adjusted_rate != None:
                by_hospital[hospital] = round(adjusted_rate, 2)
            elif adjusted_rate != None:
                by_hospital[hospital] = round((by_hospital[hospital] + adjusted_rate), 2)

    return (by_month, by_hospital)

In [11]:
summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')

({'2021-05': 192.38,
  '2021-03': 378.72,
  '2021-07': 277.67,
  '2021-06': 236.53,
  '2021-11': 229.7,
  '2021-10': 234.52,
  '2021-12': 297.87,
  '2021-04': 337.7,
  '2021-09': 160.4,
  '2021-01': 111.91,
  '2021-02': 158.55,
  '2021-08': 152.28},
 {'Missouri Baptist': 514.18,
  'SSM DePaul': 460.02,
  'SLU Hospital': 409.67,
  'Barnes Jewish': 485.7,
  'Mercy Springfield': 518.59,
  'Mercy St. Louis': 380.07})

In [13]:
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-07'] == 277.67)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[0]['2021-03'] == 378.72)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy St. Louis'] == 380.07)
assert (summarize_reimbursement('/data/visits.csv','/data/negotiated_rates.json')[1]['Mercy Springfield'] == 518.59)